In [1]:
import librosa
import soundfile as sf
import matplotlib.pyplot as plt
from IPython.display import Audio
import pandas as pd
import numpy as np 
import os
import soundfile as sf
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV, RandomizedSearchCV
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, f1_score, recall_score, precision_score, auc

In [2]:
def cut_audio_file(audio_dir_test,file,start,lenght):
# to get the audio segment and save it as segment_lenght_filename.mp3
#file = filename.mp3
# start = seconds
# lenght = seconds
    filename=os.path.join(audio_dir_test,file)
    y, sr = librosa.load(filename, sr=None)
    # Define start and end times
    end_sec = start + lenght  # 4 seconds after the start
    # Convert times to samples
    start_sample = int(start * sr)
    end_sample = int(end_sec * sr)
    # Extract the segment
    segment = y[start_sample:end_sample]
    # Save the segment
    out_filename = f'extracted_segment_{lenght}_{file}'
    sf.write(os.path.join(audio_dir_test,out_filename), segment, sr)
    

In [3]:
#get all features out of giver file 
def get_features(file_path, file_name):
    y, sr = librosa.load(file_path)
    iter_ = int(y.shape[0]/sr)
    features = []
    for i in range(iter_):
        y_seg = y[i*sr:(sr*i+sr)]
        chroma_stft = np.mean(librosa.feature.chroma_stft(y=y_seg, sr=sr))
        rms = np.mean(librosa.feature.rms(y=y_seg))
        spec_cent = np.mean(librosa.feature.spectral_centroid(y=y_seg, sr=sr))
        spec_bw = np.mean(librosa.feature.spectral_bandwidth(y=y_seg, sr=sr))
        rolloff = np.mean(librosa.feature.spectral_rolloff(y=y_seg, sr=sr))
        zcr = np.mean(librosa.feature.zero_crossing_rate(y_seg))
        mfcc = np.mean(librosa.feature.mfcc(y=y_seg, sr=sr, n_mfcc=20),axis=1)
        features.append([chroma_stft,rms,spec_cent,spec_bw,rolloff,zcr,*mfcc]) # starred expressions for mfcc to get 20 columns instead of 20 row
        # b = np.concatenate((b,a),axis=1) # ajout filename apres?
    return features
    

In [4]:
data_set = pd.read_csv('Data/DATASET-balanced.csv')


In [12]:
def make_test_features_df(audio_dir,filename):
    file_features=get_features(os.path.join(audio_dir,filename),filename)
    feature_list = []
    for segment_features in file_features:
            feature_list.append(segment_features)

    features_df = pd.DataFrame(feature_list,columns = data_set.columns[0:-1])
    return features_df


In [6]:
# Start here:
# make sure it the right path
audio_dir_test='data/DEMONSTRATION/DEMONSTRATION'
files = os.listdir(audio_dir_test)
files


['Youssef.wav',
 'Youssef_Eric Cartman.mp3',
 '.DS_Store',
 'Youssef_2_Female.mp3',
 'extracted_segment_Taylor Swift Talks Record-Breaking Midnights Album, Music Video Cameos and Easter Eggs.mp3',
 'Elise.m4a',
 'extracted_segment_100_Taylor Swift Talks Record-Breaking Midnights Album, Music Video Cameos and Easter Eggs.mp3',
 'extracted_segment_10_Taylor Swift Talks Record-Breaking Midnights Album, Music Video Cameos and Easter Eggs.mp3',
 'Taylor Swift Talks Record.mp3',
 'Youssef_2.wav',
 'linus-to-musk-DEMO.mp3',
 'Morgan Freeman.mp3',
 'Youssef_2_Male Reggaeton.mp3',
 'linus-original-DEMO.mp3']

In [33]:
#select the file to cut using files[]
file_to_cut = files[5]
file_to_cut

'extracted_segment_Taylor Swift Talks Record-Breaking Midnights Album, Music Video Cameos and Easter Eggs.mp3'

In [34]:
# cut the audio, start in second, and the lenght in seconds
cut_audio_file(audio_dir_test,file_to_cut,89,10)


In [67]:
files = os.listdir(audio_dir_test)
files

['extracted_segment_10_extracted_segment_Taylor Swift Talks Record-Breaking Midnights Album, Music Video Cameos and Easter Eggs.mp3',
 'Youssef.wav',
 'Youssef_Eric Cartman.mp3',
 'Morgan Freeman_fake.mp3',
 '.DS_Store',
 'Youssef_2_Female.mp3',
 'extracted_segment_Taylor Swift Talks Record-Breaking Midnights Album, Music Video Cameos and Easter Eggs.mp3',
 'Elise.m4a',
 'extracted_segment_100_Taylor Swift Talks Record-Breaking Midnights Album, Music Video Cameos and Easter Eggs.mp3',
 'extracted_segment_10_Taylor Swift Talks Record-Breaking Midnights Album, Music Video Cameos and Easter Eggs.mp3',
 'Taylor Swift Talks Record.mp3',
 'Morgan Freeman_real.mp3',
 'Youssef_2.wav',
 'linus-to-musk-DEMO.mp3',
 'Youssef_2_Male Reggaeton.mp3',
 'linus-original-DEMO.mp3']

In [68]:
# select file to extract
file_toextract = files[0]
file_toextract

'extracted_segment_10_extracted_segment_Taylor Swift Talks Record-Breaking Midnights Album, Music Video Cameos and Easter Eggs.mp3'

In [69]:
new_df = make_test_features_df(audio_dir_test,file_toextract)

In [70]:
new_df

,chroma_stft,rms,spectral_centroid,spectral_bandwidth,rolloff,zero_crossing_rate,mfcc1,mfcc2,mfcc3,mfcc4,...,mfcc11,mfcc12,mfcc13,mfcc14,mfcc15,mfcc16,mfcc17,mfcc18,mfcc19,mfcc20
0,0.266713,0.024175,2488.767853,2557.013024,5043.174050,0.106168,-402.563660,75.165642,-9.371619,2.917660,...,-5.777615,5.591122,-18.615765,-0.740069,-7.483478,-11.294174,-12.027924,-9.504189,-9.324254,0.393987
1,0.373345,0.025940,4048.460035,2321.003873,6252.459162,0.282227,-389.625092,36.787796,27.971003,-8.568991,...,-5.818683,2.727419,-16.109734,0.457710,-9.347790,-8.196451,-4.385269,-2.799412,-5.839867,-3.829276
2,0.389789,0.026804,4293.773383,2170.951369,6454.577637,0.311324,-358.837402,18.336519,26.020479,-5.477790,...,-5.011458,2.797537,-12.094995,-0.762202,-8.011040,-7.236790,-3.691190,-6.877252,-0.824421,-2.965919
3,0.375962,0.031323,4593.221291,2107.459574,6340.304843,0.365545,-328.825989,3.781252,12.923135,-11.903259,...,-11.460261,4.433259,-11.604218,0.576921,-5.820167,-0.672130,-3.105122,-1.135081,-1.775455,-0.462192
4,0.463718,0.015463,4560.312020,2540.055917,7316.639848,0.339733,-442.204498,22.452936,20.087536,0.011327,...,-3.454080,3.011384,-4.014338,3.774122,-2.437266,-3.924394,-2.529838,0.005294,-2.025670,-3.951313
5,0.381234,0.022823,2215.506952,2381.244658,4509.982577,0.122192,-414.437683,89.840485,13.517703,7.607273,...,-6.327803,-2.857169,-14.190909,1.462165,-3.776703,-9.829624,-6.756525,-7.122979,-7.637768,-4.460651
6,0.275166,0.029981,2957.555537,2355.386909,5215.439675,0.158780,-313.219269,60.018173,-8.165321,6.385196,...,-9.996703,6.760010,-21.714996,-1.331829,-1.017129,-12.262683,-3.193378,-8.046390,-5.552390,-5.300855
7,0.454376,0.013076,4030.790530,2358.128929,6343.485884,0.304532,-354.919098,28.704393,-12.822949,5.976503,...,-5.561951,0.791053,-9.922685,7.310260,2.251214,-6.659727,-4.908119,0.151156,-3.669740,0.247016
8,0.385579,0.026736,3869.650489,2403.439047,6411.021839,0.291038,-310.292938,44.211666,12.915974,-3.458895,...,-9.102305,2.853955,-17.968302,-2.541740,0.722498,-5.838959,-5.517236,0.088289,-5.671933,-5.467321
9,0.319002,0.035594,2157.215094,2237.772023,4329.886697,0.103771,-288.831726,94.895164,-20.275801,-7.117326,...,-14.184451,6.337558,-14.688366,-1.442739,-2.655276,-16.023396,-0.906840,-7.214269,-6.002391,-10.560570


In [71]:
file_toextract[:-4]

'extracted_segment_10_extracted_segment_Taylor Swift Talks Record-Breaking Midnights Album, Music Video Cameos and Easter Eggs'

In [72]:
new_df.to_csv(f'data/{file_toextract[:-4]}.csv',index=False)